<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/Haystack_Seminal_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách xây dựng một pipeline QA scalable với Elasticsearch**



# 1. Kiểm tra GPU runtime

In [1]:
%%bash

nvidia-smi

Mon Jul 10 05:56:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]==1.17.2

# 3. Chuẩn bị các file tài liệu

In [3]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/mcu"

fetch_archive_from_http(
    url="https://github.com/kynthesis/HaystackResearch/raw/main/mcu.zip",
    output_dir=doc_dir,
)

True

# 4. Cài đặt Elasticsearch

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [5]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [6]:
import time

time.sleep(30)

# 5. Khởi tạo ElasticsearchDocumentStore

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(
    host=os.environ.get("ELASTICSEARCH_HOST", "localhost"),
    username="",
    password="",
    index="mcu",
    similarity="dot_product",
    embedding_dim=768
)

# 6. Khởi tạo IndexingPipeline, TextConverter, và PreProcessor

In [8]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 7. Indexing các file tài liệu vào ElasticsearchDocumentStore

In [9]:
import os

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/32 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/32 [00:00<?, ?docs/s]

{'documents': [<Document: {'content': "Carol Danvers becomes one of the universe's most powerful heroes when Earth is caught in the middle of a galactic war between two alien races.\n\nAfter crashing an experimental aircraft, Air Force pilot Carol Danvers is discovered by the Kree and trained as a member of the elite Starforce Military under the command of her mentor Yon-Rogg. Six years later, after escaping to Earth while under attack by the Skrulls, Danvers begins to discover there's more to her past. With help from S.H.I.E.L.D. agent Nick Fury, they set out to unravel the truth.\n\nIn 1995, on the Kree Empire's capital planet of Hala, warrior and Starforce member Vers suffers from recurring nightmares involving an older woman. Yon-Rogg, her mentor and commander, warns her to control her abilities while the Supreme Intelligence, an organic artificial intelligence who acts as the ruler of the Kree, urges her to keep her emotions in check.\n\nWhile working with official Ronan the Accus

# 8. Khởi tạo Retriever

In [11]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

document_store.update_embeddings(retriever)

Updating embeddings:   0%|          | 0/496 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/496 [00:00<?, ? Docs/s]

# 9. Khởi tạo SemanticSearchPipeline

In [12]:
semantic_search_pipeline = Pipeline()

semantic_search_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])

# 10. Demo SearchSearchPipeline

In [13]:
result = semantic_search_pipeline.run(
    "The green guy",
    params={"Retriever": {"top_k": 5}}
)

from haystack.utils import print_documents

print_documents(result, max_text_len=200, print_name=True, print_meta=True)


Query: The green guy

{   'content': 'Bruce Banner, a scientist on the run from the U.S. Government, '
               'must find a cure for the monster he turns into whenever he '
               'loses his temper.\n'
               '\n'
               "Depicting the events after the Gamma Bomb. 'The Incredible ...",
    'meta': {   '_split_id': 0,
                '_split_overlap': [   {   'doc_id': 'b45507df283cdf1071b66a4f50a81074',
                                          'range': [963, 1106]}]},
    'name': None}

{   'content': 'He explains that his mom made him the mix tape of her favorite '
               'songs. She listens and likes it. He asks her to dance, but she '
               "doesn't trust him. He says it reminds him of an old fable "
               'about other peop...',
    'meta': {   '_split_id': 9,
                '_split_overlap': [   {   'doc_id': '8c4ab0f3001b9c62484358842a71f9ba',
                                          'range': [0, 144]},
                 

# 11. Khởi tạo Reader

In [14]:
from haystack.nodes import FARMReader

reader = FARMReader("ahotrod/albert_xxlargev1_squad2_512")

# 11. Khởi tạo ExtractiveQAPipeline

In [15]:
extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
extractive_qa_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

# 12. Demo ExtractiveQAPipeline

In [16]:
result = extractive_qa_pipeline.run(
    query="Who is the guy with the Mind Stone?",
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

from haystack.utils import print_answers

print_answers(result, details="medium", max_text_len=200)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: Who is the guy with the Mind Stone?'
'Answers:'
[   {   'answer': "'Vision'.",
        'context': '" metal man (played by Paul Bettany), who they take to '
                   "referring to as 'Vision'. Wanda is instantly smitten with "
                   'Vision and even attempts to flirt wit',
        'score': 0.9543946385383606},
    {   'answer': 'Strucker',
        'context': 'Bruce, Tony and Steve manage to penetrate the fortress. '
                   'Steve captures Strucker as he is preparing to flee. '
                   'Strucker monologues for a bit, long enough',
        'score': 0.0014318835455924273},
    {   'answer': 'Dr. Arnim Zola',
        'context': 'r. A German voice speaks and analyzes the two. The voice '
                   'belongs to Dr. Arnim Zola (Toby Jones), the HYDRA '
                   'scientist responsible for building Red Skul',
        'score': 0.0011545359157025814},
    {   'answer': 'Yondu',
        'context': 'vaporizes Ronan. Rocket colle

# 13. Khởi tạo Generator

In [17]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

# 14. Khởi tạo GenerativeQAPipeline

In [18]:
generative_qa_pipeline = Pipeline()

generative_qa_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
generative_qa_pipeline.add_node(component=reader, name="Generator", inputs=["Retriever"])

# 15. Demo GenerativeQAPipeline

In [28]:
result = extractive_qa_pipeline.run(
    query="Tell me something about Dr.Strange?",
    params={"Retriever": {"top_k": 5}}
)

from haystack.utils import print_answers

print_answers(result, details="medium", max_text_len=200)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: Tell me something about Dr.Strange?'
'Answers:'
[   {   'answer': 'a surgeon with a God complex',
        'context': 'a deadly enemy who is out to destroy it.\n'
                   '\n'
                   'Stephen Strange is a surgeon with a God complex. When he '
                   'gets in an accident and his hands are injured, he f',
        'score': 0.7264957427978516},
    {   'answer': 'brilliant but arrogant',
        'context': 'Strange and Mordo save the world?\n'
                   '\n'
                   'Sadly, Dr Stephen Strange, a brilliant but arrogant New '
                   'York City neurosurgeon on top of his game, is more '
                   'interest',
        'score': 0.7238306999206543},
    {   'answer': 'brilliant but arrogant New York City neurosurgeon on top of '
                  'his game, is more interested in fame and his perfect '
                  'success rate than saving lives',
        'context': ', a brilliant but arrogant New York City ne

# 16. Khởi tạo Agent bằng API Key OpenAI

In [46]:
from getpass import getpass

api_key_prompt = "Enter OpenAI API key:"
api_key = getpass(api_key_prompt)

from haystack.agents import Agent
from haystack.nodes import PromptNode

prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=api_key, stop_words=["Observation:"])
agent = Agent(prompt_node=prompt_node)

Enter OpenAI API key:··········


# 17. Cung cấp Tool cho Agent

In [ ]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/mcu"

fetch_archive_from_http(
    url="https://github.com/kynthesis/HaystackResearch/raw/main/mcu.zip",
    output_dir=doc_dir,
)

from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

import os

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

In [77]:
from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1", use_gpu=True
)
document_store.update_embeddings(retriever=retriever)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
mcu_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

from haystack.utils import print_answers

result = mcu_qa.run("What is the nickname of the man who turned The Mad Titan into ash?")

print_answers(result, "minimum")

Updating Embedding:   0%|          | 0/496 [00:00<?, ? docs/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the nickname of the man who turned The Mad Titan into ash?'
'Answers:'
[   {   'answer': 'Thanos',
        'context': 'The mad titan Thanos has begun his quest to obtain all six '
                   'Infinity Stones, which will give him the power to wipe out '
                   'half of all life in the universe'},
    {   'answer': 'Mighty Thor',
        'context': "'s surprise - inexplicably wields his magical hammer, "
                   'Mjolnir, as the Mighty Thor. Together, they embark upon a '
                   'harrowing cosmic adventure to uncover '},
    {   'answer': 'THE INCREDIBLE HULK',
        'context': 'life as Bruce Banner or the creature he could permanently '
                   'become: THE INCREDIBLE HULK.\n'
                   '\n'
                   'Over the opening credits, we see how the Hulk was born. '
                   'Bruce '},
    {   'answer': 'Mysterio',
        'context': 'ates, and after watching a news report on him, they start '
 

In [47]:
from haystack.agents import Tool

search_tool = Tool(
    name="MCU_QA",
    pipeline_or_node=mcu_qa,
    description="useful for when you need to answer questions related to the Marvel's MCU.",
    output_variable="answers",
)

agent.add_tool(search_tool)

In [76]:
result = agent.run("What is the nickname of the man who turned The Mad Titan into ash?")

print(result["transcript"].split("---")[0])


Agent zero-shot-react started with {'query': 'What is the nickname of the man who turned The Mad Titan into ash?', 'params': None}
 find out who The Mad Titan is.
Tool: MCU_QA
Tool Input: Who is The Mad Titan?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Observation: Thanos
Thought:  Now that I know who The Mad Titan is, I can find out the nickname of the man who turned him into ash.
Tool: MCU_QA
Tool Input: Who turned Thanos into ash?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Observation: Stark
Thought:  Now that I found the identity of the man, I can figure out his nickname.
Tool: MCU_QA
Tool Input: What is the nickname of Stark?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Observation: Iron Man
Thought:  With this information, I can answer the question.
Final Answer: Iron Man find out who The Mad Titan is.
Tool: MCU_QA
Tool Input: Who is The Mad Titan?

Observation: Thanos
Thought: Now that I know who The Mad Titan is, I can find out the nickname of the man who turned him into ash.
Tool: MCU_QA
Tool Input: Who turned Thanos into ash?


Observation: Stark
Thought: Now that I found the identity of the man, I can figure out his nickname.
Tool: MCU_QA
Tool Input: What is the nickname of Stark?


Observation: Iron Man
Thought: With this information, I can answer the question.
Final Answer: Iron Man
